# Transitions NREM-REM

In [59]:
import sys
import itertools

import numpy as np
import pandas as pd
import scipy.signal
from scipy.signal import savgol_filter
from scipy.stats import zscore

from tqdm import tqdm

import neuroseries as nts


import time
import matplotlib.pyplot as plt
# %matplotlib qt

import seaborn as sns 
import bk.load
import bk.compute
import bk.plot

import os
%load_ext memory_profiler
import cProfile

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [2]:
import matplotlib as mpl
# COLOR = 'white'
# mpl.rcParams['text.color'] = COLOR
# mpl.rcParams['axes.labelcolor'] = COLOR
# mpl.rcParams['axes.edgecolor'] = COLOR
# mpl.rcParams['xtick.color'] = COLOR
# mpl.rcParams['ytick.color'] = COLOR
# mpl.rcParams['figure.facecolor'] = '#282C34'
# mpl.rcParams['axes.facecolor'] = '#282C34'
mpl.rcParams['svg.fonttype'] = 'none'
# %matplotlib qt

# Utils Functions

In [3]:
def transition(states, template,epsilon = 0):
    '''
    author: BK
    states : dict of nts.Interval_set
    template : list of state.
    epsilon : int, will drop any transition in which there is an epoch shorter than epsilon 's'
    This function will find transition that match the template 
    '''
    
    long = pd.DataFrame()
    for s,i in states.items():
        i['state'] = s
        long = pd.concat((i,long))
        del i['state']
    order = np.argsort(long.start)
    long = long.iloc[order]
    
    transition_times = []
    transition_intervals = []
    for i,s in enumerate(long.state):
        tmp = list(long.state[i:i+len(template)])
        if tmp == template:
            tmp_transition = long.iloc[i:i+len(template)]
#             print(d.iloc[i:i+len(template)])
            length = (tmp_transition.end - tmp_transition.start)/1_000_000
            if np.any(length.values<epsilon): 
                continue
            tmp_pre = np.array(tmp_transition.end[:-1])
            tmp_post = np.array(tmp_transition.start[1:])
            tmp_times = np.mean([tmp_pre,tmp_post],0)
            
            
            transition_intervals.append([tmp_transition.start.iloc[0],tmp_transition.end.iloc[-1]])
            transition_times.append(tmp_times)
    
    transition_times = np.array(transition_times)
    transition_intervals = np.array(transition_intervals)
    transition_intervals = nts.IntervalSet(start = transition_intervals[:,0],end = transition_intervals[:,1],force_no_fix = True)
    return transition_intervals,transition_times

## Computing

In [4]:
def sanity_check(neurons,states,intervals):
    plt.ioff()
    f = plt.figure(figsize=(12,4),dpi = 300)
    ax = plt.subplot(3,1,1)
    plt.suptitle(f'Rat : {bk.load.rat} Day : {bk.load.day} - {bk.load.session}')
    
    for k,s in states.items():
        if   k == 'Rem'    : c = 'orange'
        elif k == 'sws'    : c = 'grey'
        elif k == 'wake'   : c = 'pink'
        elif k == 'drowsy' : c = 'green'
            
        bk.plot.intervals(s,c)

    
    plt.subplot(3,1,2,sharex = ax)
    bk.plot.intervals(intervals)
    
    plt.subplot(3,1,3,sharex = ax)
    bk.plot.rasterPlot(neurons)
    plt.tight_layout()
    plt.savefig(f'/home/billel/figs/{bk.load.session}.jpeg')
    f.clear()
    plt.close(f)
#     lfp = bk.load.loadLFP()
#     plt.subplot(4,1,4,sharex = ax)
#     plt.plot(lfp.as_units('s'))
    

In [5]:
def main(local_path = 'Rat08/Rat08-20130713',arg = None,plot = False, save = False):
    #Variables :
    if arg == None:
        transition_interest = [['Rem','sws','Rem']] #Template of transitions to follow.
        bin_epochs = [12,30,12]
    else:
        transition_interest = [arg[0]]
        bin_epochs = arg[1]
    
    bk.load.current_session_linux(local_path = local_path)
    neurons,metadata = bk.load.loadSpikeData(bk.load.path)
    states = bk.load.states()
    
    #Check transitions : 
    

    for i,transition_name in enumerate(transition_interest): #For each kind of transitions
        print(f'Computing {transition_name} transitions')
        intervals,timing = transition(states,transition_name,epsilon=50) #Find the timing of the transitions
        
        
        print(f'Found {len(intervals)} transitions')
        transition_activity = []
        for event,t in zip(intervals.iloc,timing): #For each transitions
            epochs = np.array([[event.start,t[0]],[t[0],t[1]],[t[1],event.end]])
            #epochs = np.array([(event.start,t[0]),(t[0],event.end)],dtype = np.int64)
            epochs = nts.IntervalSet(start = epochs[:,0],end = epochs[:,1])
            #Creates intervals for each state of the transitions events.


    #         binned = np.array(shape = (252,np.sum(bin_epochs)))
    #         binned = np.empty(shape = (252,np.sum(bin_epochs),len(intervals)+1))
            binned = np.empty(shape = (len(neurons),1))
            for i,epoch in enumerate(epochs.as_units('s').iloc):
                start = epoch.start
                end = epoch.end
                nbins = bin_epochs[i]
                _,b = bk.compute.binSpikes(neurons,start = start,stop = end,nbins=nbins)
                b = b/((end-start)/nbins) # Converting to firing rates

                binned = np.hstack((binned,b))
            binned = binned[:,1:]
            transition_activity.append(binned)
        
        transition_activity = np.array(transition_activity)
        transition_activity = np.moveaxis(transition_activity,0,2)
        if plot:
            print(np.mean(transition_activity[metadata.Region == "BLA"],2).T.shape)
            y_bla = scipy.stats.zscore(np.mean(transition_activity[metadata.Region == "BLA"],2).T,0)
            y_hpc = scipy.stats.zscore(np.mean(transition_activity[metadata.Region == "Hpc"],2).T,0)

            plt.figure()
            plt.plot(y_bla,'green')
            plt.plot(y_hpc,'blue')

            plt.title(f'Rat:{bk.load.rat}, Day{bk.load.day} {bk.load.session}')
        print(os.getcwd())
        # sanity_check(neurons,states,intervals)
        return np.array(transition_activity),metadata

In [7]:
bk.load.current_session_linux()

Rat : 8 on day : 6
Working with session Rat08-20130713 @ /home/billel/Data/GG-Dataset/Rat08/Rat08-20130713


True

In [8]:
arg = [['sws','Rem','sws'],[30,12,30]]
NREM_REM_NREM = bk.load.batch(main,arg,verbose=False,linux=True)
arg = [['Rem','sws','Rem'],[12,30,12]]
REM_NREM_REM = bk.load.batch(main,arg,verbose=False,linux=True)

100%|██████████| 61/61 [01:04<00:00,  1.06s/it]

Batch finished in 64.72613191604614
Some session were not processed correctly
['Rat08-20130712', 'Rat08-20130716', 'Rat08-20130717', 'Rat08-20130718', 'Rat08-20130720', 'Rat08-20130722', 'Rat09-20140324', 'Rat09-20140325', 'Rat09-20140328', 'Rat09-20140329', 'Rat09-20140331', 'Rat09-20140401', 'Rat09-20140402', 'Rat09-20140404', 'Rat09-20140407', 'Rat10-20140622', 'Rat10-20140628', 'Rat10-20140701', 'Rat10-20140702', 'Rat10-20140703', 'Rat10-20140707', 'Rat11-20150313', 'Rat11-20150315', 'Rat11-20150316', 'Rat11-20150317', 'Rat11-20150321', 'Rat11-20150325', 'Rat11-20150327', 'Rat11-20150328', 'Rat11-20150330', 'Rat11-20150331', 'Rat11-20150402', 'Rat11-20150403']
54.09836065573771 %


 ## Merging

In [149]:
def merge(batch,name,discard):
    transitions_activity = []
    metadata = []
    n_transitions = 0
    n_sessions = 0
    for i,b in batch.items():
        #if i not in discard:
         if True:
            transitions_activity.append(np.mean(b[0],2))
            metadata.append(b[1])
            n_transitions += b[0].shape[2]
            n_sessions += 1 
    merged = {'name':name,'average_transition':np.vstack(transitions_activity),'metadata':pd.concat(metadata),'n_transitions':n_transitions,'n_sessions':n_sessions}
    merged['metadata'].Type[merged['metadata'].Region == 'CeCM'] = 'Int'
    return merged

In [150]:
discard = ['Rat08-20130718','Rat09-20140408','Rat09-20140409','Rat10-20140619','Rat10-20140620','Rat10-20140622']

In [151]:
m_NREM_REM_NREM = merge(NREM_REM_NREM,'NREM_REM_NREM',discard)
m_REM_NREM_REM = merge(REM_NREM_REM,'REM_NREM_REM',discard)

## Plotting

In [152]:
%matplotlib qt

In [168]:
stru = ['Hpc','CeCM','BLA','Pir']
types = ['Pyr','Int']
transitions = [m_NREM_REM_NREM]
for trans in transitions:
    for s in stru:
        fig,ax = plt.subplot_mosaic('''AB
                                       AC''')
        stru_mask = trans['metadata'].Region == s
        for t in types:
            types_mask = trans['metadata'].Type == t
            stru_type_activity = trans['average_transition'][(stru_mask) & types_mask]
            if np.any(stru_type_activity) :
                print(s+t)
                pre_fr = np.mean(zscore(stru_type_activity[:,0:30],0),1)
                post_fr = np.mean(zscore(stru_type_activity[:,42:],0),1)
                
                print(scipy.stats.wilcoxon(pre_fr,post_fr))
                a = np.array([pre_fr,post_fr]).T
                v = pd.DataFrame(a,columns=[f'{t}Pre',f'{t}Post'])
                ax['A'].plot(zscore(np.mean(stru_type_activity.T,1)))
                ax['A'].axvspan(15,30)
                ax['A'].axvspan(42,42+15)
                ax['A'].axvspan(30,42,color = 'orange')
                if t == 'Pyr':
                    plt.sca(ax['B'])
                    sns.violinplot(data = v,inner = 'point')
                else:
                    plt.sca(ax['C'])
                    sns.violinplot(data = v,inner = 'point')

                plt.suptitle(trans['name']+s)
            
            plt.title(f'{t}')

HpcPyr
WilcoxonResult(statistic=164057.0, pvalue=0.45658392885319543)
HpcInt
WilcoxonResult(statistic=3186.0, pvalue=0.5684921227549036)
CeCMInt
WilcoxonResult(statistic=87562.0, pvalue=0.9613028206686308)
BLAPyr
WilcoxonResult(statistic=301924.0, pvalue=0.209615298054312)
BLAInt
WilcoxonResult(statistic=7135.0, pvalue=0.642062222443402)
PirPyr
WilcoxonResult(statistic=280840.0, pvalue=0.6494352151216134)
PirInt
WilcoxonResult(statistic=2535.0, pvalue=0.8908732749937845)


In [126]:
for r,i in enumerate(['Hpc','CeCM','BLA','Pir']):
    plt.figure()
    for t,j in enumerate(['Pyr','Int'],1):
        stru_type_activity = transitions_activity[(metadata.Region == i) & (metadata.Type == j)]
        plt.subplot(1,2,t)
        if np.any(stru_type_activity):
#             order = np.argsort(np.mean(stru_type_activity[:,30:41],1)/np.mean(stru_type_activity,1))
            plt.title(f'Stru : {i} type : {j}')
            plt.imshow(stru_type_activity[:,:],aspect='auto');
            plt.clim([-5,5])
    plt.tight_layout()

NameError: name 'transitions_activity' is not defined

In [141]:
metadata[metadata.Region == 'Hpc']

,Rat,Day,Shank,Id,Region,Type
0,8,1,4,2,Hpc,Pyr
1,8,1,4,3,Hpc,Pyr
2,8,1,4,4,Hpc,Int
3,8,1,4,5,Hpc,Pyr
4,8,1,4,6,Hpc,Pyr
...,...,...,...,...,...,...
73,11,20,18,15,Hpc,Int
74,11,20,20,2,Hpc,Pyr
75,11,20,20,3,Hpc,Pyr
76,11,20,20,4,Hpc,Pyr


In [67]:
bk.load.current_session_linux()

Rat : 8 on day : 6
Working with session Rat08-20130713 @ /home/billel/Data/GG-Dataset/Rat08/Rat08-20130713


True

In [68]:
states = bk.load.states()

In [69]:
transition(states,['sws'])

(          start          end
 0    1448000000   1697000000
 1    1777000000   1926000000
 2    2104000000   2429000000
 3    2826000000   3275000000
 4    3383000000   3841000000
 5    3888000000   3916000000
 6    3970000000   3992000000
 7    4024000000   4529000000
 8    4568000000   4648000000
 9    5044000000   5426000000
 10   5489000000   6069000000
 11   6368000000   6629000000
 12   6970000000   7311000000
 13   7375000000   7759000000
 14   7788000000   7909000000
 15   8485000000   8797000000
 16   8887000000   9333000000
 17  15516000000  16145000000
 18  16436000000  16765000000
 19  16801000000  17259000000
 20  17364000000  17429000000
 21  17497000000  17669000000
 22  17689000000  17764000000
 23  17791000000  17810000000
 24  17852000000  18076000000
 25  18132000000  18528000000
 26  18663000000  18979000000
 27  19086000000  19167000000
 28  19187000000  19448000000
 29  19546000000  19771000000
 30  19818000000  20363000000
 31  20867000000  21286000000
 32  21359

In [4]:
# transition_interest = [('Rem','sws'),('sws','Rem'),('Rem','wake')]
# s = "ABC"
# fig,ax = plt.subplot_mosaic('ABC')
# for ii,i in enumerate(transition_interest):
#     binned = []
#     for event,trans_time in zip(transitions[i].as_units('s').iloc,times[i].as_units('s').index.values):
#         print(trans_time)
#         print('he')
#         start = event.start
#         end = event.end
        
#         if (end-trans_time)<50 or (trans_time-start) <50: continue
# #         if (end-trans_time) > 180: end = trans_time+120
# #         if (trans_time-start) > 180: start = trans_time-120
        
        
        
#         t_pre,b_pre = bk.compute.binSpikes(neurons,start = start,stop = trans_time,nbins=30)
#         t_post,b_post = bk.compute.binSpikes(neurons,start = trans_time,stop = end,nbins=12)
#         b_pre = b_pre/((trans_time-start)/30)
#         b_post = b_post/((end-trans_time)/12)
#         t = np.hstack((t_pre,t_post))
#         b = np.hstack((b_pre,b_post))
        
# #         plt.figure()
# #         plt.plot(t,b.T)
# #         plt.axvline(trans_time,color = 'red')
#         binned.append(b)
#     ax[s[ii]].plot(np.mean(binned,0).T)
    
# plt.suptitle('Transition')